In [13]:
import pandas as pd
import attr
import numpy as np
import geopandas as gpd
import os
from shapely.affinity import scale
from shapely.geometry import Point, Polygon
import plotly.express as px
import matplotlib.pyplot as plt
from pandas import DataFrame as DF, Series as S
from scipy.stats import skew
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from peakutils import baseline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.cross_decomposition import PLSRegression  # noqa: E402
from sklearn.ensemble import (  # noqa: E402
    ExtraTreesRegressor,  # noqa: E402
)
from scipy.signal import savgol_filter
import funcy
import random
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from caching import Cache
from sklearn.cluster import KMeans
import seaborn as sns
from functools import lru_cache, reduce
from custom_split import GroupKFold, StratifiedKFold, KFold
import scipy


DEV = False
DEBUG = True
cached = Cache(DEV, debug=DEBUG)

In [14]:
SEED = 123123
N_JOBS = 9

def remove_baseline(vs) -> np.ndarray:
    base = np.apply_along_axis(lambda x: baseline(x, 2), 0, vs)
    base = np.mean(base, axis=0)
    vs = vs - base
    return vs

def gradients(x: np.ndarray) -> np.ndarray:
    return np.apply_along_axis(np.gradient, 1, x)

def smoothing(x: np.ndarray, window_size=51, order=3) -> np.ndarray:
    return savgol_filter(x, window_size, order, axis=1)


def xtrees_factory():
    return ExtraTreesRegressor(n_jobs=N_JOBS, random_state=SEED, 
                               **{'n_estimators': 300, 'max_features': 'auto', 'max_depth': 50})


def standard_clean(raw):
    return transform_specs(raw, funcy.compose(smoothing, gradients, remove_baseline))

def spec_cols_mask(df):
    mask = df.columns.str.match("[0-9]+\.[0-9]+")
    return mask

def spec_cols(df):
    return df.iloc[:, spec_cols_mask(df)]

def spec_cols_names(df):
    return df.columns[spec_cols_mask(df)]


def transform_specs(df, f):
    df = df.copy()
    x = spec_cols(df)
    x_transformed = DF(f(x), columns=df.columns[spec_cols_mask(df)])
    return pd.concat([df.loc[:, ~spec_cols_mask(df)].reset_index(drop=True), x_transformed], axis=1)

@cached("cv_score_vnir")
def cv_score(df, 
             analyte,
             stratify=None,
             model_factory=xtrees_factory,
             random_state=SEED, 
             adverse=False,
             train_subset_size=1.0):
    
    n = len(df.index)
    if isinstance(train_subset_size, int):
        train_subset_size = train_subset_size / n
    
    df = df.sample(frac=1, random_state=random_state) # random shuffle
    model = model_factory()
    X = spec_cols(df)
    y = df[analyte]
    if stratify:
        if isinstance(stratify, (str, tuple, list, MultiStrat, ClusterStrat)):
            stratify = strat_f(stratify)
        else:
            raise ValueError("functional stratification no longer supported")
        stratify_col = stratify(df)
        if adverse:
            gkf = GroupKFold(n_splits=S(stratify_col).nunique(), train_subset_frac=train_subset_size)
            cv = gkf.split(X, stratify_col, stratify_col)
        else:
            skf = StratifiedKFold(n_splits=5, train_subset_frac=train_subset_size)
            cv = skf.split(X, stratify_col)
    else:
        cv = KFold(train_subset_frac=train_subset_size)
    return cross_val_score(model, X, y, scoring="r2", cv=cv, n_jobs=N_JOBS)

def get_n_strata(n_cols):
    if DEV:
        n = 5
    else:
        n = 10 # TODO change to 5 andrecompute cache!

    n = int(n ** (1 / n_cols))
    return max(n, 2)


class MultiStrat(dict):
    @classmethod
    def from_list(cls, l):
        return cls({k: get_n_strata(len(l)) for k in l})
    
    
    
    


class ClusterStrat(tuple):
    pass


class OrderedStrat(list):
    pass

MS = MultiStrat
CS = ClusterStrat


def parse_strat(strat):
    if isinstance(strat, (MS, CS, OrderedStrat)):
        return strat
    elif isinstance(strat, str):
        return MultiStrat({strat: get_n_strata(1)})
    elif isinstance(strat, (tuple, list)):
        raise ValueError("Please use multi stratification explicitly")
    else:
        assert False, strat
        

    
def encode_tuple(t, sizes):
    assert len(t) == len(sizes)
    t = np.array(t)
    for i, size in enumerate(sizes):
        t[i+1:] *= size

    return sum(t)
        
        
def strat_f(strat):
    strat = parse_strat(strat)
    if isinstance(strat, ClusterStrat):
        return clustered_strat_f(strat)
    
    if isinstance(strat, OrderedStrat):
        return ordered_strat_f(strat)
    
    discretizer_c = lambda n: KBinsDiscretizer(n_bins=n, encode="ordinal", strategy="quantile")
    discretize = lambda n, x: discretizer_c(n).fit_transform(x.values.reshape(-1, 1)).flatten().astype(int)

    def quantile_strat_f(df):
        discretize_df = lambda col_n: discretize(col_n[1], df[col_n[0]])
        ys_discrete = list(map(discretize_df, strat.items()))
        ys_discrete = np.stack(ys_discrete, axis=1)
        return np.apply_along_axis(lambda x: encode_tuple(x, strat.values()), axis=1, arr=ys_discrete)


    return quantile_strat_f


def clustered_strat_f(strat):
    def _strat_f(df):
        km = KMeans(n_clusters=8)
        km.fit(df[list(strat)])
        return km.labels_
        
    return _strat_f


def discretize_series(x, n_bins):
    s = S(x)
    
    bin_size = len(s) / n_bins
    s = s.sort_values()
    for i in range(n_bins):
        s[int(i*bin_size): int((i + 1)*bin_size)] = i

    return s.sort_index()

def encode_cols_as_ordinal(*cols):
    cols = list(map(S, cols))
    df = pd.concat(cols, axis=1)
    unique_rows = df.drop_duplicates().reset_index(drop=True)
    return df.merge(unique_rows.reset_index(), sort=False, how="left")["index"].values

def ordered_strat_f(strat):
    ST_COL = "__STRAT_COL__"
    ST_COL2 = "__STRAT_COL2__"
    strat = list(strat)
    
    def recursive_strat_f(df, col_n_tuples):
        df = df.copy()
        if not col_n_tuples:
            return S(0, index=df.index).values
        
        col, n_bins = col_n_tuples.pop(0)
        df[ST_COL] = recursive_strat_f(df, col_n_tuples)
        subdfs = []
        for _, subdf in df.groupby(ST_COL):
            subdf = subdf.copy()
            subdf[ST_COL2] = discretize_series(subdf[col], n_bins)
            subdfs.append(subdf)
            

        df = pd.concat(subdfs, axis=0).sort_index()

        return encode_cols_as_ordinal(df[ST_COL], df[ST_COL2])

    
    def strat_f(df):
        return recursive_strat_f(df, strat)
    
    return strat_f

In [15]:
wet = pd.read_csv('/media/Data/jp_dir/sis/usda/extracted_metadata/c_622_measured_vnir.csv')

In [16]:
dry = pd.read_csv('/media/Data/jp_dir/sis/usda/vnir_spectres/vnirlib.csv')

In [17]:
@cached("preprocess_dry")
def preprocess_dry(df):
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.iloc[1:, :]
    df = df.reset_index()
    df = df.merge(wet, left_on="index", right_on="scan_path_name")
    df.columns = df.columns.astype(str)

    return df


vnir_df = preprocess_dry(dry)




getting cached cache/prod/preprocess_dry/df:shape:(2151, 69140)##b93641580d8e92fc0dc89c382859f95566add85593222ed431b59ec353cc2b21


In [18]:
C = vnir_df.sample(10000)

In [19]:
C

,index,350.0,351.0,352.0,353.0,354.0,355.0,356.0,357.0,358.0,...,2496.0,2497.0,2498.0,2499.0,2500.0,smp_id,scan_path_name,value_of_measured_property,soil_property_name,units_of_measure
7939,97114MD01.asd,13.440278,14.475272,15.401972,15.782609,16.105050,17.799791,19.114040,19.910538,20.529756,...,1438.536151,1403.924014,1368.916917,1333.691709,1298.536725,97114,97114MD01.asd,3.848301,"Carbon, Total NCS",% wt
7186,137003CF03.asd,10.853419,11.566726,12.175624,12.785781,13.318303,14.097071,15.171807,16.176907,16.945429,...,225.712523,219.934538,213.805809,207.883968,201.332618,137003,137003CF03.asd,0.445790,"Carbon, Total NCS",% wt
20242,143050CF01.asd,20.251948,21.507867,22.531937,24.086333,25.694280,26.909204,27.929247,29.809532,31.778198,...,1040.500090,1014.974707,989.180487,963.489345,937.499926,143050,143050CF01.asd,0.142515,"Carbon, Total NCS",% wt
48861,181130MD01.asd,6.711766,6.882932,7.149249,7.457144,7.761932,8.242017,8.370838,8.348284,8.574403,...,873.571013,845.221144,816.618163,789.666070,762.975248,181130,181130MD01.asd,0.739258,"Carbon, Total NCS",% wt
60832,198249MD01.asd,8.451436,8.976681,9.571116,9.949665,10.173780,11.084784,11.377221,11.753894,12.618790,...,1199.416295,1160.284820,1120.494424,1084.250183,1047.935342,198249,198249MD01.asd,0.050062,"Carbon, Total NCS",% wt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60099,197343MD01.asd,12.960219,13.976315,14.674714,14.758218,14.769170,16.704672,16.925164,17.042919,18.416359,...,1178.269234,1141.162407,1101.974981,1065.765446,1030.798646,197343,197343MD01.asd,0.303674,"Carbon, Total NCS",% wt
16740,138053CF01.asd,24.272200,25.898171,27.531962,29.211914,30.848211,32.846514,35.118210,36.850970,38.545981,...,951.697859,928.476342,904.970243,881.684359,858.443533,138053,138053CF01.asd,4.720814,"Carbon, Total NCS",% wt
52932,217844MD01.asd,5.620887,5.687879,5.868703,6.645793,7.732320,7.303147,7.436392,7.826312,8.035179,...,1471.829033,1427.137374,1380.942349,1337.928707,1295.891121,217844,217844MD01.asd,1.234452,"Carbon, Total NCS",% wt
16699,137987MD01.asd,12.124705,12.939422,13.791114,14.368192,14.991234,15.815475,16.800096,17.665551,18.659833,...,1299.595711,1270.227253,1241.078914,1211.220142,1181.056164,137987,137987MD01.asd,0.342561,"Carbon, Total NCS",% wt


In [20]:
clean_C = standard_clean(C)

In [21]:
cv_score(clean_C, "value_of_measured_property")

array([0.91327102, 0.89484438, 0.91454969, 0.91238779, 0.9032734 ])

In [32]:
def truncate_freqs(df):
    s_cols_mask = spec_cols_mask(df)
    wet_part = df.loc[:, ~s_cols_mask].reset_index(drop=True)
    dry_part = df.loc[:, s_cols_mask]
    wavelengths = dry_part.columns.astype(float)
    wls_mask = (wavelengths >= 400) & (wavelengths <= 1000)
    truncated_dry = dry_part.loc[:, wls_mask].reset_index(drop=True)
    return pd.concat([wet_part, truncated_dry], axis=1)


def report_vnir_lc(df, analyte, sizes, seeds, stratify=False):
    
    stratify = analyte if stratify else None

    all_scores = []
    for seed in seeds:
        print(f"evaluating with seed {seed}...")
        
        for size in sizes:
            subdf = df.sample(size, random_state=seed)
            truncated = truncate_freqs(subdf)
            scores = cv_score(subdf, analyte, random_state=seed, stratify=stratify)
            truncated_scores = cv_score(truncated, analyte, random_state=seed, stratify=stratify)
            all_scores += [(s, size, "no") for s in scores]
            all_scores += [(s, size, "yes") for s in truncated_scores]
            
            
    return DF(all_scores, columns=["score", "size", "truncated"]).groupby(["size", "truncated"]).mean()

In [35]:
sizes = [100, 200, 300, 500, 1000, 2000]
seeds = range(SEED, SEED + 5)
report_vnir_lc(clean_C, "value_of_measured_property", sizes, seeds, stratify=True)

evaluating with seed 123123...
getting cached cache/prod/cv_score_vnir/df:shape:(100, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#5929a71d286f7d07fd52e2a025f820deca822493ca265058c6681fea1cc82f65
getting cached cache/prod/cv_score_vnir/df:shape:(100, 607)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#c3b35d788385c01af1f1947f04fc69a984d1d2ee3601749ac88d732bc379bd37
getting cached cache/prod/cv_score_vnir/df:shape:(200, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#203fb56c5337baf5935dc49ff371f3ccaa428d31081515cbc9c524978108c2b7
getting cached cache/prod/cv_score_vnir/df:shape:(200, 607)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#91ba9cf0e4dba926575f70bc17244489a71dc1339828e521f3902824265bf320
getting cached cache/prod/cv_score_vnir/df:shape:(300, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_

score
size truncated          
100  no         0.344176
     yes        0.124150
200  no         0.625750
     yes        0.444521
300  no         0.751944
     yes        0.600535
500  no         0.808115
     yes        0.676985
1000 no         0.858345
     yes        0.727532
2000 no         0.877741
     yes        0.749754

In [33]:
report_vnir_lc(clean_C, "value_of_measured_property", sizes, seeds, stratify=True)

evaluating with seed 123123...
getting cached cache/prod/cv_score_vnir/df:shape:(200, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#203fb56c5337baf5935dc49ff371f3ccaa428d31081515cbc9c524978108c2b7
getting cached cache/prod/cv_score_vnir/df:shape:(300, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#ac451fa2e0fbc56edb8dc37647f85e7f07b7c638f821f1114e9859d9efe9bf52
getting cached cache/prod/cv_score_vnir/df:shape:(500, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#46f009638b7f5de2df8e396d875614c64f7158c7aea1e9d6140004d681668ffa
getting cached cache/prod/cv_score_vnir/df:shape:(1000, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measured_property#33821b0c45510f682b2794285a9a6cb25f0a04cab6810334494c114a25036ee4
getting cached cache/prod/cv_score_vnir/df:shape:(2000, 2157)-value_of_measured_property#random_state:123123-stratify:value_of_measu

score
size truncated          
200  no         0.746505
     yes        0.494376
300  no         0.780991
     yes        0.600190
500  no         0.824221
     yes        0.667466
1000 no         0.867135
     yes        0.708345
2000 no         0.879425
     yes        0.740536

In [ ]:
truncated_C = truncate_freqs(clean_C)

In [ ]:
cv_score(truncated_C, "value_of_measured_property")

In [ ]:
np.mean([0.80451762, 0.79666925, 0.80615234, 0.81824066, 0.80082898])